# Imports

 Cover fundamentals of tensors using TensorFlow.
 
- Introduction to tensors
- Getting information from tensors
- Manipulating tensors
- Tensors and NumPy
- Using @tf.function (a way to speed up python)
- Using GPUs with TensorFlow (or TPUs)
- Exercises to try for yourself


## Introduction to Tensors

In [4]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub

In [3]:
print(tf.__version__)

2.14.0


In [6]:
# Creating Tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [7]:
# Check number of dimensions of tensor, ndim
scalar.ndim

0

In [8]:
# Create a vector
vector = tf.constant([10,10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [9]:
vector.ndim

1